### Importing all the important Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from dotenv import dotenv_values
from sqlalchemy import create_engine
import sklearn
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import pairwise
from sklearn.metrics import accuracy_score
import statsmodels.formula.api as sm
from scipy.sparse import csr_matrix
import os
import streamlit as sl

### Reading different datasets 

### Filtering Movie titles

### Reading sorted Data

In [ ]:
n= pd.read_csv("../data/movie_sorted.csv")

In [ ]:
n.tail()

### Reading the rating files for movieid 1 and cleaning the file 

In [ ]:
df = pd.read_csv("../data/mv_0000001.txt", names=["customerid","rating","date"] )

In [ ]:
df.head()

### checking the first cleaned rating file 

In [ ]:
df= pd.read_csv("../data/movieid1_rating.csv")

In [ ]:
df.head()

### Creating a function to read and clean all the rating files

### Creating and checking a test version of concatinated rating files

In [ ]:
df_s= pd.read_csv("../data/ratings_small.csv", names=["customerid","rating","date", "movieid"])

In [ ]:
df_s["date"] = pd.to_datetime(df_s["date"], format = "%Y-%m-%d")

In [ ]:
df_s.info()

### Creating the large file containing all the ratings

### Reading and transforming the Qualify  file to add the Machine Learing results

### Reading the cleaned qualify file 

In [ ]:
qualifier= pd.read_csv("../data/qualify.csv")

In [ ]:
qualifier

### Cleaning the probe file to compare the results of qualify dataset created with help of machine learning Algorithms

### Reading the cleaned probe file

In [ ]:
probability = pd.read_csv("../data/probe.csv")

In [ ]:
probability

## NEXT TASKS

#### Filter the data to create a better machine learning program

In [ ]:
small= pd.read_csv("../data/ratings_small.csv" )
small.head()

In [11]:
movies = pd.read_csv("../data/movie_sorted.csv")
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17770 entries, 0 to 17769
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieid  17770 non-null  int64 
 1   year     17770 non-null  int64 
 2   title    17770 non-null  object
dtypes: int64(2), object(1)
memory usage: 416.6+ KB


In [ ]:
ratings = pd.read_csv("../../thymestamps-working-folder/data/selected-ratings.csv")
ratings.head()

In [ ]:
ratings.info()

In [ ]:
ratings["movieid"].value_counts()

In [ ]:
ratings['customerid'].nunique(), ratings['movieid'].nunique(), ratings['movieid'].max()

In [ ]:
user = csr_matrix((ratings['rating'], (ratings['movieid'], ratings['customerid'])))

#### like pivot_table but much more memory efficient

In [ ]:
user.shape

In [ ]:
# customerid vector
user[1,:].todense()

In [ ]:
# movieid vector
user[:,1].todense().shape

In [ ]:
UU = pairwise.euclidean_distances(user)
print(UU.shape)

In [ ]:
sns.heatmap(UU, cmap='coolwarm')

#### NEw file

In [2]:
r =  pd.read_csv("../../thymestamps-working-folder/data/filtered-ratings.csv")
r.head()

,customerid,rating,movieid,title
0,2407893,4.0,6974,The Usual Suspects
1,1673319,5.0,6974,The Usual Suspects
2,602,5.0,6974,The Usual Suspects
3,2604811,4.0,6974,The Usual Suspects
4,1329723,5.0,6974,The Usual Suspects


In [22]:
r["movieid"].value_counts()

1905     120787
11283    113216
15124    113000
15107    109851
5317     108621
          ...  
14021      1283
17096      1234
761        1230
9044       1219
11819      1165
Name: movieid, Length: 5264, dtype: int64

In [3]:
u = csr_matrix((r['rating'], (r['movieid'], r['customerid'])))

In [4]:
# initialize the unsupervised model
model = NearestNeighbors(metric='cosine')


In [6]:
# fit it to the user-item matrix
model.fit(u)

NearestNeighbors(metric='cosine')

In [8]:
# top-10 most similar users for the user with the id 1 (id1 is included in the neighborhood!)
model.kneighbors(u[1,:], n_neighbors=20)

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1.]]),
 array([[11852, 11866, 11836, 11851, 11850, 11849, 11848, 11847, 11846,
         11845, 11844, 11843, 11842, 11841, 11840, 11839, 11838, 11853,
         11865, 11864]], dtype=int64))

In [9]:
u[1,:].shape

(1, 2649430)

In [23]:
liked = [1905,11283,15124,15107,5317,14021,17096,761,9044,11819]

In [24]:
# new user vector
u_vec = np.repeat(0, 2649430)
u_vec.shape

# fill in some ratings
u_vec[liked] = 5
u_vec.shape

(2649430,)

In [25]:
distances, user_ids = model.kneighbors([u_vec], n_neighbors=10)

In [26]:
user_ids

array([[ 9475,  7168, 12139,  9169, 12978,  2408,   840,   907, 12440,
         3331]], dtype=int64)

In [ ]:
neighborhood = r.set_index('movieid').loc[user_ids[0]]
neighborhood

In [29]:
recommendations = neighborhood.groupby('movieid')['rating'].sum().sort_values(ascending=False)
recommendations

movieid
3331     22438.0
907      17412.0
2408     16012.0
12978    13584.0
12440    13241.0
12139    11580.0
840      11094.0
9169      9133.0
7168      7768.0
9475      7445.0
Name: rating, dtype: float64

In [30]:
liked

[1905, 11283, 15124, 15107, 5317, 14021, 17096, 761, 9044, 11819]

In [31]:
# filter out movies that the user allready watched
item_filter = ~recommendations.index.isin(liked)
recommendations = recommendations.loc[item_filter]

In [32]:
# top 10 recommendations
movies.loc[recommendations.head(10).index]

,movieid,year,title
movieid,,,
3331,3332,1997,Spike and Mike's Sick & Twisted Festival of An...
907,908,1981,Sense and Sensibility
2408,2409,1976,Robin and Marian
12978,12979,2002,The Adventures of Ociee Nash
12440,12441,1980,The Martian Chronicles
12139,12140,1983,48 Hrs.
840,841,1994,There Goes My Baby
9169,9170,2001,Ghost World
7168,7169,1998,Palmetto


In [34]:
recommended_movie_ids = recommendations.head(10).index
recommended_movie_ids

Int64Index([3331, 907, 2408, 12978, 12440, 12139, 840, 9169, 7168, 9475], dtype='int64', name='movieid')

In [ ]:
relevant_items

In [35]:
x = r[["rating","movieid"]].values
x

array([[4.000e+00, 6.974e+03],
       [5.000e+00, 6.974e+03],
       [5.000e+00, 6.974e+03],
       ...,
       [5.000e+00, 6.974e+03],
       [4.000e+00, 6.974e+03],
       [5.000e+00, 6.974e+03]])

In [36]:
y = r["customerid"].values
y

array([2407893, 1673319,     602, ..., 1208146,  873100, 2221430],
      dtype=int64)

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

In [38]:
sc = StandardScaler()

In [39]:
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)


In [40]:
KNN = KNeighborsClassifier()

In [41]:
KNN.fit(x_train, y_train)

KNeighborsClassifier()

In [ ]:
pred_y = KNN.predict(x_test)

In [ ]:
pred_y

In [ ]:
y_test

In [ ]:
accuracy_score(y_test, pred_y)

In [12]:
# collect the movie ids for later

# for calculating recommendations
liked_items = [11283,550,4306,1905,14691,14410,9016,5657,14021,761,11819]
# for testing the recommender after getting some recommendations
relevant_items = []

### Deciding a Machine learning technique
#### Neighborhood-based Collaborative Filtering( Nearest Neighbors)
#### Neighborhood Components Analysis
#### nearest neighbors classifier

In [ ]:
# which metrics can we use
sorted(sklearn.neighbors.VALID_METRICS_SPARSE['brute'])

In [ ]:
# initialize the unsupervised model
model = NearestNeighbors(metric='cosine')


In [ ]:
# fit it to the user-item matrix
model.fit(user)

In [ ]:
# top-10 most similar users for the user with the id 1 (id1 is included in the neighborhood!)
model.kneighbors(user[1,:], n_neighbors=20)

In [ ]:
user[1,:].shape

In [ ]:
# new user vector
user_vec = np.repeat(0, 17770)
user_vec.shape

# fill in some ratings
user_vec[liked_items] = 5
user_vec.shape

In [ ]:
# find the neighborhood
distances, user_ids = model.kneighbors([user_vec], n_neighbors=10)

In [ ]:
user_ids

In [ ]:
# find the ratings for the neighbors
neighborhood = ratings.set_index('customerid').loc[user_ids[0]]
neighborhood

In [ ]:
recommendations = neighborhood.groupby('movieid')['rating'].sum().sort_values(ascending=False)
recommendations

In [ ]:
liked_items

In [ ]:
# filter out movies that the user allready watched
item_filter = ~recommendations.index.isin(liked_items)
recommendations = recommendations.loc[item_filter]

In [ ]:
# top 10 recommendations
movies.loc[recommendations.head(10).index]

In [ ]:
recommended_movie_ids = recommendations.head(10).index
recommended_movie_ids

In [ ]:
relevant_items

#### Test the algorithm

#### maybe create some visualisations to check the data distribution 

#### Add ratings in Qualify dataset and compare with probe to check the accuracy

#### Create a Predictor system which takes 3 inputs from user and provides list of related movies 

### Create a GUI using Streamlit